In [ ]:
import os
from os.path import join as ojoin
sim_dir = ojoin('simulation_results') 

In [ ]:
def extract_coverage(fpath):
    total_coverage = 0
    with open(fpath) as f:
        for line in f:
            line = line.strip()
            if not line: continue
            total_coverage += float(line)
    return total_coverage

In [ ]:
def get_coverage_dict():
    coverage_dict = dict()
    
    for method in ('tim', 'simpath', 'imm', 'celf'):
        output_dir = ojoin(sim_dir, method)
        coverage_list = []
        for graph_number in range(20):
            coverage = extract_coverage(ojoin(output_dir, 'output_%d.txt'%graph_number))
            if coverage == 0:
                print('ERROR')
            coverage_list.append(coverage)
        coverage_dict[method] = coverage_list

    for nsamples in (10, 100, 200, 500, 1000):
        output_dir = ojoin(sim_dir, 'mip', 'n%d'%nsamples)
        coverage_list = []
        for graph_number in range(20):
            coverage = extract_coverage(ojoin(output_dir, 'output_%d.txt'%graph_number))
            if coverage == 0:
                print('ERROR')
            coverage_list.append(coverage)
        coverage_dict[('mip', nsamples)] = coverage_list  

    return coverage_dict

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def draw(values, labels, colors, ylabel, fpath, ylim=None, figsize=None):
    if figsize is None:
        figsize=(3.5, 2)
    plt.figure(figsize=figsize)
    means = [np.mean(v) for v in values]
    stds = [np.std(v) for v in values]
    xpos = range(len(labels))

    bars = plt.bar(xpos, means, yerr=stds, width=0.5, align='center', capsize=4)
    for i in xpos:
        bars[i].set_color(colors[i])
    _ = plt.xticks(xpos, labels, rotation=0, fontsize=10)
    if ylim:
        _ = plt.ylim(ylim)
    _ = plt.ylabel(ylabel, fontsize=10)
    _ = plt.savefig(fpath, bbox_inches='tight')

In [ ]:
coverage_dict = get_coverage_dict()

sample_range = (10, 100, 200, 500, 1000)
labels = ['celf', 'simpath', 'imm', 'tim']
for nsamples in sample_range:
    labels.append('MIP\n%d'%nsamples)

colors = ['#ff85d0'] * 4 + ['#8fffa0'] * 5

keys = ['celf', 'simpath', 'imm', 'tim'] 
for nsamples in sample_range:
    keys.append(('mip', nsamples))

values = [np.array(coverage_dict[key]) for key in keys]
fpath = 'all_methods.pdf'
draw(values, labels, colors, 'Coverage', fpath, ylim=(65, 80), figsize=(7, 2.5))